

# 1.Download, Install Hadoop and configure

In [1]:
# The default JVM available at /usr/lib/jvm/java-11-openjdk-amd64/  works for Hadoop
# But gives errors with Hive https://stackoverflow.com/questions/54037773/hive-exception-class-jdk-internal-loader-classloadersappclassloader-cannot
# Hence this JVM needs to be installed
!apt-get update > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


# If there is an error in this cell, it is very likely that the version of hadoop has changed
# Download the latest version of Hadoop and change the version numbers accordingly
#wget -q https://downloads.apache.org/hadoop/common/hadoop-3.3.0/hadoop-3.3.0.tar.gz
#!wget -q https://downloads.apache.org/hadoop/common/hadoop-3.3.2/hadoop-3.3.2.tar.gz
#!wget  https://downloads.apache.org/hadoop/common/hadoop-3.3.2/hadoop-3.3.2.tar.gz
!wget -q https://downloads.apache.org/hadoop/common/hadoop-3.3.5/hadoop-3.3.5.tar.gz
# Unzip it
# the tar command with the -x flag to extract, -z to uncompress, -v for verbose output, and -f to specify that we’re extracting from a file
#!tar -xzf hadoop-3.3.2.tar.gz
!tar -xzf hadoop-3.3.5.tar.gz
#copy  hadoop file to user/local
#!mv  hadoop-3.3.2/ /usr/local/
!mv  hadoop-3.3.5/ /usr/local/

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
#To set java path, go to /usr/local/hadoop-3.3.0/etc/hadoop/hadoop-env.sh then
#. . . export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64/ . . .
#we have used a simpler alternative route using os.environ - it works

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"   # default is changed
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"
# make sure that the version number is as downloaded
#os.environ["HADOOP_HOME"] = "/usr/local/hadoop-3.3.0/"
#os.environ["HADOOP_HOME"] = "/usr/local/hadoop-3.3.2/"
os.environ["HADOOP_HOME"] = "/usr/local/hadoop-3.3.5/"

In [3]:
# Add Hadoop BIN to PATH
# Get the current_path from output of previous command
#current_path = '/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin'
#current_path = '/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin'
#new_path = current_path+':/usr/local/hadoop-3.3.2/bin/'
#os.environ["PATH"] = new_path
#!echo $PATH

current_path = os.getenv('PATH')
#new_path = current_path+':/usr/local/hadoop-3.3.0/bin/'
#new_path = current_path+':/usr/local/hadoop-3.3.2/bin/'
new_path = current_path+':/usr/local/hadoop-3.3.5/bin/'
os.environ["PATH"] = new_path
!echo $PATH

/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/usr/local/hadoop-3.3.5/bin/


In [38]:
# 1.2 Test Hadoop installation

In [4]:
#Running Hadoop - Test RUN, not doing anything at all
!/usr/local/hadoop-3.3.5/bin/hadoop
# UNCOMMENT the following line if you want to make sure that Hadoop is alive!
!hadoop

Usage: hadoop [OPTIONS] SUBCOMMAND [SUBCOMMAND OPTIONS]
 or    hadoop [OPTIONS] CLASSNAME [CLASSNAME OPTIONS]
  where CLASSNAME is a user-provided Java class

  OPTIONS is none or any of:

buildpaths                       attempt to add class files from build tree
--config dir                     Hadoop config directory
--debug                          turn on shell script debug mode
--help                           usage information
hostnames list[,of,host,names]   hosts to use in worker mode
hosts filename                   list of hosts to use in worker mode
loglevel level                   set the log4j level for this command
workers                          turn on worker mode

  SUBCOMMAND is one of:


    Admin Commands:

daemonlog     get/set the log level for each daemon

    Client Commands:

archive       create a Hadoop archive
checknative   check native Hadoop and compression libraries availability
classpath     prints the class path needed to get the Hadoop jar and the re

In [39]:
#1.3 Formatting HDFS, Starting Hadoop DFS, and Uploading Data to HDFS

In [9]:
!hadoop/bin/hdfs namenode -format
!hadoop/sbin/start-dfs.sh
!hadoop/bin/hdfs dfs -mkdir -p /input
!hadoop/bin/hdfs dfs -put /content/shakespeare.txt /input/
!hadoop/bin/hdfs dfs -ls /input

2025-03-26 09:50:14,870 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = 2c8192260009/172.28.0.12
STARTUP_MSG:   args = [-format]
STARTUP_MSG:   version = 3.3.6
STARTUP_MSG:   classpath = /content/hadoop/etc/hadoop:/content/hadoop/share/hadoop/common/lib/netty-codec-stomp-4.1.89.Final.jar:/content/hadoop/share/hadoop/common/lib/netty-transport-rxtx-4.1.89.Final.jar:/content/hadoop/share/hadoop/common/lib/commons-math3-3.1.1.jar:/content/hadoop/share/hadoop/common/lib/netty-resolver-dns-native-macos-4.1.89.Final-osx-x86_64.jar:/content/hadoop/share/hadoop/common/lib/netty-codec-redis-4.1.89.Final.jar:/content/hadoop/share/hadoop/common/lib/animal-sniffer-annotations-1.17.jar:/content/hadoop/share/hadoop/common/lib/hadoop-shaded-guava-1.1.1.jar:/content/hadoop/share/hadoop/common/lib/jetty-http-9.4.51.v20230217.jar:/content/hadoop/share/hadoop/common/lib/netty-codec-xml-4.1.89.Final.ja

# 2. Defining the mapper script (map.py)

In [27]:
%%writefile map.py
#!/usr/bin/env python3
import sys
import re

for line in sys.stdin:
    fields = line.strip().split(",")

    # Remove punctuation and convert to lowercase
    words = re.findall(r'\b\w+\b', line.lower())
    for word in words:
        print(f"{word}\t1")

Overwriting map.py


In [28]:
!cat shakespeare.txt | python3 map.py

Streaming output truncated to the last 5000 lines.
good	1
allow	1
you	1
are	1
my	1
all	1
the	1
world	1
and	1
i	1
must	1
strive	1
to	1
know	1
my	1
shames	1
and	1
praises	1
from	1
your	1
tongue	1
none	1
else	1
to	1
me	1
nor	1
i	1
to	1
none	1
alive	1
that	1
my	1
steeled	1
sense	1
or	1
changes	1
right	1
or	1
wrong	1
in	1
so	1
profound	1
abysm	1
i	1
throw	1
all	1
care	1
of	1
others	1
voices	1
that	1
my	1
adder	1
s	1
sense	1
to	1
critic	1
and	1
to	1
flatterer	1
stopped	1
are	1
mark	1
how	1
with	1
my	1
neglect	1
i	1
do	1
dispense	1
you	1
are	1
so	1
strongly	1
in	1
my	1
purpose	1
bred	1
that	1
all	1
the	1
world	1
besides	1
methinks	1
are	1
dead	1
since	1
i	1
left	1
you	1
mine	1
eye	1
is	1
in	1
my	1
mind	1
and	1
that	1
which	1
governs	1
me	1
to	1
go	1
about	1
doth	1
part	1
his	1
function	1
and	1
is	1
partly	1
blind	1
seems	1
seeing	1
but	1
effectually	1
is	1
out	1
for	1
it	1
no	1
form	1
delivers	1
to	1
the	1
heart	1
of	1
bird	1
of	1
flower	1
or	1
shape	1
which	1
it	1
doth	1
latch	1
of	1
his	1
q

# 3. Defining the reducer script ( reduce.py)

In [33]:

%%writefile reduce.py
#!/usr/bin/env python3
import sys
from collections import defaultdict

word_counts = defaultdict (int)

for line in sys.stdin:
    word, count = line.strip().split("\t")
    word_counts[word] += + int(count)


for word, count in sorted(word_counts.items()):
    print(f"{word}\t{count}")

Overwriting reduce.py


In [34]:
!hadoop fs -ls /input

Found 1 items
-rw-r--r--   1 root root      94275 2025-03-26 09:50 /input/shakespeare.txt


# 4. Running hadoop streaming job with mapper and reducer script

In [35]:
!hadoop/bin/hadoop jar hadoop/share/hadoop/tools/lib/hadoop-streaming-*.jar \
    -input /input/shakespeare.txt \
    -output /output \
    -mapper "python3 map.py" \
    -reducer "python3 reduce.py"

2025-03-26 10:18:12,583 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-03-26 10:18:12,813 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-03-26 10:18:12,813 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2025-03-26 10:18:12,837 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2025-03-26 10:18:13,236 INFO mapred.FileInputFormat: Total input files to process : 1
2025-03-26 10:18:13,265 INFO mapreduce.JobSubmitter: number of splits:1
2025-03-26 10:18:13,722 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1243022132_0001
2025-03-26 10:18:13,722 INFO mapreduce.JobSubmitter: Executing with tokens: []
2025-03-26 10:18:13,999 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
2025-03-26 10:18:14,004 INFO mapreduce.Job: Running job: job_local1243022132_0001
2025-03-26 10:18:14,016 INFO mapred.LocalJobRunner: OutputCommitter set in config null
2025-03

# 5. Results

In [36]:
!hadoop fs -ls /output

Found 2 items
-rw-r--r--   1 root root          0 2025-03-26 10:18 /output/_SUCCESS
-rw-r--r--   1 root root      28330 2025-03-26 10:18 /output/part-00000


In [37]:
!hadoop fs -cat /output/part-*

a	164
abhor	2
abide	2
able	1
about	1
above	4
absence	5
absent	4
abundance	4
abundant	1
abuse	3
abused	1
abuses	1
abysm	1
accents	1
acceptable	1
acceptance	1
accessary	1
accident	1
accidents	1
account	4
accumulate	1
accuse	2
accusing	1
achieve	1
acknowledge	1
acquaintance	3
acquainted	2
act	1
action	3
active	1
actor	1
add	4
added	2
adder	1
addeth	1
adding	1
addition	2
adieu	1
adjunct	2
admire	1
admired	1
admiring	1
admit	1
admitted	1
adonis	1
adore	1
adulterate	1
advance	1
advantage	3
adverse	1
advised	1
advocate	1
afar	1
affable	1
affairs	2
affections	1
afford	1
affords	2
afloat	1
afresh	1
after	11
afterwards	1
again	10
against	18
age	17
ages	1
aggravate	1
ah	7
aid	2
air	4
alack	3
alas	2
alchemy	2
alien	1
alike	1
alive	2
all	120
allayed	1
allege	1
allow	2
almost	3
aloft	1
alone	19
already	1
alt	1
alter	1
alteration	1
altered	2
alters	2
although	9
always	1
am	35
amazeth	1
ambush	1
amen	1
amends	1
amis	1
amiss	2
among	3
an	17
anchored	1
and	490
anew	3
angel	5
anger	1
angry	1
annexed	1
an